In [1]:
import numpy as np
import rebound
import matplotlib.pyplot as plt
import json
import pandas as pd
import os
import tqdm

In [2]:
ss_bodies = ["Sun", "Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"]

In [3]:
attrs = ['x', 'y', 'z']#, 'vx', 'vy', 'vz']

In [4]:
def growth_linear(final_mass, N_outputs):
    return np.mgrid[initial_sun_mass:final_mass:N_outputs*1j]

def growth_exponential(final_mass, N_outputs):
    ratio = np.log10(final_mass/initial_sun_mass)
    return 10**np.mgrid[0.0:ratio:N_outputs*1j]

final_masses = [0.001, 0.1, 2, 10, 25, 100, 1000]
growth_funcs = {
    "linear": growth_linear,
    "exponential": growth_exponential,
}

In [5]:

if os.path.exists("ss_initial_conditions.bin"):
    sim = rebound.Simulation("ss_initial_conditions.bin")
else:
    sim = rebound.Simulation()
    sim.units = ('day', 'AU', 'Msun')
    for body in ss_bodies:
        sim.add(body)
    sim.save_to_file("ss_initial_conditions.bin")
initial_sun_mass = sim.particles[0].m

In [6]:
def output_orbits(funcname, final_mass):
    tmax = 365 * 10
    dt = 1
    N_outputs = tmax // dt
    mass = growth_funcs["exponential"](final_mass, N_outputs)

    output = {"time": [], "Msun": []}
    output.update({f"{b}_{attr}": [] for b in ss_bodies for attr in attrs})
    for i in tqdm.tqdm(range(N_outputs)):
        sim.particles[0].m = mass[i]
        sim.integrate(sim.t + dt)
        for b, p in zip(ss_bodies, sim.particles):
            for attr in attrs:
                output[f"{b}_{attr}"].append(getattr(p, attr))
        output['time'].append(sim.t)
        output['Msun'].append(sim.particles[0].m)
    df = pd.DataFrame(output).set_index("time")
    df.to_csv(f"orbits_{funcname}_{final_mass:04d}.csv", float_format = "%0.4g")

In [8]:
for func in ["linear", "exponential"]:
    for final_mass in [2, 5, 10, 25, 50, 100, 1000]:
        print(f"{func}=>{final_mass}")
        output_orbits(func, final_mass)

linear=>2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3650/3650 [00:00<00:00, 7975.03it/s]


linear=>5


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3650/3650 [00:00<00:00, 8111.96it/s]


linear=>10


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3650/3650 [00:01<00:00, 3638.14it/s]


exponential=>100


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3650/3650 [00:00<00:00, 8378.16it/s]


exponential=>1000


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3650/3650 [00:00<00:00, 8582.61it/s]
